In [ ]:
# Script créé par Julien Caugant, bibliothécaire-assistant à Aix-Marseille Université, sous licence GNU-GPL
import PyPDF2, requests as rq, json, pandas as pd
import urllib.request
import os
import re, time
keyword = ["GitHub","Github"]
k_found = []
hal_id = []
submit_by = []

In [ ]:
url = "https://api.archives-ouvertes.fr/search/?q=*:*&wt=json&rows=500&fq=submitType_s:file&fq=structId_i:198056&fq=submittedDateY_i:2024&fl=uri_s,contributorFullName_s"

In [ ]:
req = rq.get(url)
req = req.json()

In [ ]:
for i in range(0, len(req['response']['docs'])):
    try :
        pdf = req['response']['docs'][i]['uri_s']+"/document"
        sub = req['response']['docs'][i]['contributorFullName_s']
        urllib.request.urlretrieve(pdf, "doc.pdf")
        read_pdf = open('doc.pdf', 'rb')
        try :
            pdfReader = PyPDF2.PdfReader(read_pdf)
            for j in range(0,len(pdfReader.pages)):
                pageObj = pdfReader.pages[j]
                text=(pageObj.extract_text())
                text=text.split(",")
                for word in text :
                    for k in keyword :
                        if k in word :
                            print(f"Le terme {k} a été trouvé dans le document {pdf}")
                            k_found.append(k)
                            hal_id.append(pdf)
                            submit_by.append(sub)
        except :
            continue
    except :
        continue
columns = ['Terme trouvé','Identifiant HAL','Déposant']
df = pd.DataFrame(columns=columns)
df['Terme trouvé'], df['Identifiant HAL'], df['Déposant'] = k_found, hal_id, submit_by
df.drop_duplicates(subset ="Identifiant HAL", keep = 'first', inplace=True)

                
            
    

In [ ]:
df

In [ ]:
df.to_csv("result.csv")

In [ ]:
pdf